In [4]:
import plaid
from plaid.model import *


# Plaid API Credentials (Use your own keys from Plaid Dashboard)
PLAID_CLIENT_ID = "67e843b405d1170022eda3f8"
PLAID_SECRET = "3df2ac4f4597c9ba98af368be97696"
PLAID_ENV = plaid.Environment.Sandbox  # Use Sandbox, Development, or Production

# Initialize Plaid API Client
configuration = plaid.Configuration(
    host=PLAID_ENV,
    api_key={
        "clientId": PLAID_CLIENT_ID,
        "secret": PLAID_SECRET
    }
)
api_client = plaid.ApiClient(configuration)
client = plaid_api.PlaidApi(api_client)  # <--- This creates the client

In [5]:
from plaid.api import plaid_api
from plaid.model.sandbox_public_token_create_request import SandboxPublicTokenCreateRequest
from plaid.model.products import Products
from plaid.model.country_code import CountryCode
from plaid.model.plaid_error import PlaidError
from plaid.exceptions import ApiException
# Create a sandbox public token
def create_sandbox_token():
    request = SandboxPublicTokenCreateRequest(
        institution_id="ins_109508",  
        initial_products=[Products("transactions")],  
    )
    
    response = client.sandbox_public_token_create(request)

    return response["public_token"]

# Get a sandbox token
public_token = create_sandbox_token()
print("Sandbox Public Token:", public_token)

Sandbox Public Token: public-sandbox-a240223e-75e2-4214-8537-b2ca1b444aab


In [6]:
from plaid.model.item_public_token_exchange_request import ItemPublicTokenExchangeRequest

# Exchange public token for access token
def exchange_sandbox_token(public_token):
    request = ItemPublicTokenExchangeRequest(public_token=public_token)
    response = client.item_public_token_exchange(request)
    return response["access_token"]

access_token = exchange_sandbox_token(public_token)
print("Sandbox Access Token:", access_token)

Sandbox Access Token: access-sandbox-618b1f5b-1c55-40c5-b90e-df6daa1cc953


In [ ]:
import pandas as pd
user_access_tokens = [access_token]
def get_transactions(client, user_access_tokens):
    start_date = (datetime.now() - timedelta(days=30)).date()
    end_date = datetime.now().date()

    all_transactions = []

    for token in user_access_tokens:
        request = TransactionsGetRequest(
            access_token=token,
            start_date=start_date,
            end_date=end_date,
            options={"count": 100}
        )
        response = client.transactions_get(request)

        for transaction in response["transactions"]:
            all_transactions.append({
                "account_id": transaction["account_id"],
                "amount": transaction["amount"],
                "date": transaction["date"],
                "name": transaction["name"],
                "merchant_name": transaction.get("merchant_name", "Unknown"),
                "category": transaction.get("category", ["Uncategorized"])[0]
            })

    return pd.DataFrame(all_transactions)

get_transactions(client,user_access_tokens)


In [11]:
from plaid.model.transactions_get_request import TransactionsGetRequest
from datetime import datetime, timedelta
import pandas as pd
start_date = (datetime.now() - timedelta(days=30)).date()
end_date = datetime.now().date()
all_transactions = []
# Get fake transactions
user_access_tokens = [access_token]

for token in user_access_tokens:  # Loop through multiple banks
    request = TransactionsGetRequest(
        access_token=token,
        start_date=start_date,
        end_date=end_date,
        options={"count": 5}  # Fetch 100 transactions per bank
    )
    response = client.transactions_get(request)
    
    # Extract relevant fields from each transaction
    for transaction in response["transactions"]:
        all_transactions.append({
            "account_id": transaction["account_id"],
            "amount": transaction["amount"],
            "date": transaction["date"],
            "name": transaction["name"],
            "merchant_name": transaction.get("merchant_name", "Unknown"),
            "category": transaction.get("category", ["Uncategorized"])[0],
            "bank": transaction["account_id"]  # You may map this to a bank name
        })

# Convert to Pandas DataFrame
df_transactions = pd.DataFrame(all_transactions)

# Display first 5 rows
print(df_transactions.head())
def get_sandbox_transactions(access_token):
    request = TransactionsGetRequest(
        access_token=access_token,
        start_date=start_date,
        end_date=end_date,
        options={"count": 55}  # Limit to 5 transactions
    )
    response = client.transactions_get(request)
    return response["transactions"]

                              account_id  amount        date  \
0  oelAy5Vj85SbDrNqy1dDFbRGd8alBrCoxMbDw  2078.5  2025-03-29   
1  oelAy5Vj85SbDrNqy1dDFbRGd8alBrCoxMbDw   500.0  2025-03-29   
2  oelAy5Vj85SbDrNqy1dDFbRGd8alBrCoxMbDw   500.0  2025-03-29   
3  Lem8PVZ3nVSL457ka934FkwW8GVLNMIkvQEMB    25.0  2025-03-20   
4  1gznWlpxKlIQmqxR3kKmtXb9jQqNpGcpLKrRD     5.4  2025-03-20   

                           name merchant_name        category  \
0     AUTOMATIC PAYMENT - THANK          None         Payment   
1                           KFC           KFC  Food and Drink   
2          Madison Bicycle Shop          None           Shops   
3  CREDIT CARD 3333 PAYMENT *//          None         Payment   
4        Uber 063015 SF**POOL**          Uber          Travel   

                                    bank  
0  oelAy5Vj85SbDrNqy1dDFbRGd8alBrCoxMbDw  
1  oelAy5Vj85SbDrNqy1dDFbRGd8alBrCoxMbDw  
2  oelAy5Vj85SbDrNqy1dDFbRGd8alBrCoxMbDw  
3  Lem8PVZ3nVSL457ka934FkwW8GVLNMIkvQEMB  
4  1gznWl

In [12]:
df_transactions

,account_id,amount,date,name,merchant_name,category,bank
0,oelAy5Vj85SbDrNqy1dDFbRGd8alBrCoxMbDw,2078.5,2025-03-29,AUTOMATIC PAYMENT - THANK,None,Payment,oelAy5Vj85SbDrNqy1dDFbRGd8alBrCoxMbDw
1,oelAy5Vj85SbDrNqy1dDFbRGd8alBrCoxMbDw,500.0,2025-03-29,KFC,KFC,Food and Drink,oelAy5Vj85SbDrNqy1dDFbRGd8alBrCoxMbDw
2,oelAy5Vj85SbDrNqy1dDFbRGd8alBrCoxMbDw,500.0,2025-03-29,Madison Bicycle Shop,None,Shops,oelAy5Vj85SbDrNqy1dDFbRGd8alBrCoxMbDw
3,Lem8PVZ3nVSL457ka934FkwW8GVLNMIkvQEMB,25.0,2025-03-20,CREDIT CARD 3333 PAYMENT *//,None,Payment,Lem8PVZ3nVSL457ka934FkwW8GVLNMIkvQEMB
4,1gznWlpxKlIQmqxR3kKmtXb9jQqNpGcpLKrRD,5.4,2025-03-20,Uber 063015 SF**POOL**,Uber,Travel,1gznWlpxKlIQmqxR3kKmtXb9jQqNpGcpLKrRD
